## Bias scan using Multi-Dimensional Subset Scan (MDSS)

"Identifying Significant Predictive Bias in Classifiers" https://arxiv.org/abs/1611.08292

The goal of bias scan is to identify a subgroup(s) that has significantly more predictive bias than would be expected from an unbiased classifier. There are $\prod_{m=1}^{M}\left(2^{|X_{m}|}-1\right)$ unique subgroups from a dataset with $M$ features, with each feature having $|X_{m}|$ discretized values, where a subgroup is any $M$-dimension
Cartesian set product, between subsets of feature-values from each feature --- excluding the empty set. Bias scan mitigates this computational hurdle by approximately identifing the most statistically biased subgroup in linear time (rather than exponential).


We define the statistical measure of predictive bias function, $score_{bias}(S)$ as a likelihood ratio score and a function of a given subgroup $S$. The null hypothesis is that the given prediction's odds are correct for all subgroups in $\mathcal{D}$:

$$H_{0}:odds(y_{i})=\frac{\hat{p}_{i}}{1-\hat{p}_{i}}\ \forall i\in\mathcal{D}.$$

The alternative hypothesis assumes some constant multiplicative bias in the odds for some given subgroup $S$:

$$H_{1}:\ odds(y_{i})=q\frac{\hat{p}_{i}}{1-\hat{p}_{i}},\ \text{where}\ q>1\ \forall i\in S\ \mathrm{and}\ q=1\ \forall i\notin S.$$

In the classification setting, each observation's likelihood is Bernoulli distributed and assumed independent. This results in the following scoring function for a subgroup $S$:

\begin{align*}
score_{bias}(S)= & \max_{q}\log\prod_{i\in S}\frac{Bernoulli(\frac{q\hat{p}_{i}}{1-\hat{p}_{i}+q\hat{p}_{i}})}{Bernoulli(\hat{p}_{i})}\\
= & \max_{q}\log(q)\sum_{i\in S}y_{i}-\sum_{i\in S}\log(1-\hat{p}_{i}+q\hat{p}_{i}).
\end{align*}
Our bias scan is thus represented as: $S^{*}=FSS(\mathcal{D},\mathcal{E},F_{score})=MDSS(\mathcal{D},\hat{p},score_{bias})$.

where $S^{*}$ is the detected most anomalous subgroup, $FSS$ is one of several subset scan algorithms for different problem settings, $\mathcal{D}$ is a dataset with outcomes $Y$ and discretized features $\mathcal{X}$, $\mathcal{E}$ are a set of expectations or 'normal' values for $Y$, and $F_{score}$ is an expectation-based scoring statistic that measures the amount of anomalousness between subgroup observations and their expectations.

Predictive bias emphasizes comparable predictions for a subgroup and its observations and Bias scan provides a more general method that can detect and characterize such bias, or poor classifier fit, in the larger space of all possible subgroups, without a priori specification.

In [1]:
import itertools

from IPython.display import Markdown, display
import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric 
from aif360.metrics.mdss_classification_metric import MDSSClassificationMetric
from aif360.metrics.mdss.ScoringFunctions.Bernoulli import Bernoulli

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

We'll demonstrate scoring a subset and finding the most anomalous subset with bias scan using the compas dataset.

We can specify subgroups to be scored or scan for the most anomalous subgroup. Bias scan allows us to decide if we aim to identify bias as `higher` than expected probabilities or `lower` than expected probabilities. Depending on the favourable label, the corresponding subgroup may be categorized as priviledged or unprivileged.

In [2]:
dataset_orig = load_preproc_data_compas()

female_group = [{'sex': 1}]
male_group = [{'sex': 0}]

The dataset has the categorical features one-hot encoded so we'll modify the dataset to convert them back 
to the categorical featues because scanning one-hot encoded features may find subgroups that are not meaningful e.g., a subgroup with 2 race values. 

In [3]:
dataset_orig_df = pd.DataFrame(dataset_orig.features, columns=dataset_orig.feature_names)

age_cat = np.argmax(dataset_orig_df[['age_cat=Less than 25', 'age_cat=25 to 45', 
                                     'age_cat=Greater than 45']].values, axis=1).reshape(-1, 1)
priors_count = np.argmax(dataset_orig_df[['priors_count=0', 'priors_count=1 to 3', 
                                          'priors_count=More than 3']].values, axis=1).reshape(-1, 1)
c_charge_degree = np.argmax(dataset_orig_df[['c_charge_degree=M', 'c_charge_degree=F']].values, axis=1).reshape(-1, 1)

features = np.concatenate((dataset_orig_df[['sex', 'race']].values, age_cat, priors_count,
                           c_charge_degree, dataset_orig.labels), axis=1)
feature_names = ['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']

In [4]:
df = pd.DataFrame(features, columns=feature_names + ['two_year_recid'])
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,two_year_recid
0,0.0,0.0,1.0,0.0,1.0,1.0
1,0.0,0.0,0.0,2.0,1.0,1.0
2,0.0,1.0,1.0,2.0,1.0,1.0
3,1.0,1.0,1.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,1.0,0.0


### Training
We'll create a structured dataset and then train a simple classifier to predict the probability of the outcome

In [5]:
from aif360.datasets import StandardDataset
dataset = StandardDataset(df, label_name='two_year_recid', favorable_classes=[0],
                 protected_attribute_names=['sex', 'race'],
                 privileged_classes=[[1], [1]],
                 instance_weights_name=None)

In [6]:
dataset_orig_train, dataset_orig_test = dataset.split([0.7])

In [7]:
metric_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=male_group,
                             privileged_groups=female_group)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train.mean_difference())
metric_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=male_group,
                             privileged_groups=female_group)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test.mean_difference())


Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.166769
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.063884


It shows that overall Females in the dataset have a lower observed recidivism them Males.

If we train a classifier, the model is likely to pick up this bias in the dataset

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(dataset_orig_train.features, dataset_orig_train.labels.flatten())

LogisticRegression(random_state=0)

In [9]:
clf.classes_

array([0., 1.])

predictions should reflect the probability of a favorable outcome (i.e. no recidivism).

In [10]:
dataset_bias_test_prob = clf.predict_proba(dataset_orig_test.features)[:, 0]

In [11]:
df = pd.DataFrame(dataset_orig_test.features, columns=dataset_orig_test.feature_names)
df['observed'] = pd.Series(dataset_orig_test.labels.flatten(), index=df.index)
df['probabilities'] = pd.Series(dataset_bias_test_prob, index=df.index)
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,observed,probabilities
0,0.0,0.0,1.0,0.0,1.0,0.0,0.691642
1,0.0,0.0,1.0,2.0,1.0,1.0,0.292973
2,0.0,1.0,2.0,2.0,1.0,1.0,0.485283
3,0.0,1.0,2.0,1.0,0.0,0.0,0.705340
4,0.0,0.0,0.0,1.0,1.0,1.0,0.328067


We'll the create another structured dataset as the classified dataset by assigning the predicted probabilities to the scores attribute

In [12]:
dataset_bias_test = dataset_orig_test.copy()
dataset_bias_test.scores = dataset_bias_test_prob
dataset_bias_test.labels = dataset_orig_test.labels

### Bias scoring

We'll create an instance of the MDSS Classification Metric and assess the a priori defined privileged and unprivileged groups; females and males respectively.

In [13]:
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=male_group,
                                           privileged_groups=female_group)

In [14]:
female_privileged_score = mdss_classified.score_groups(privileged=True)
female_privileged_score

4.681783313737739

In [15]:
male_unprivileged_score = mdss_classified.score_groups(privileged=False)
male_unprivileged_score

1.6172475495806111

It appears there is some multiplicative increase in the odds of recidivism for females and a multiplicative decrease in the odds for males. We can alternate our assumptions of privileged and unprivileged groups to see that there is no bias in that direction.

In [16]:
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=female_group,
                                           privileged_groups=male_group)

In [17]:
male_privileged_score = mdss_classified.score_groups(privileged=True)
male_privileged_score

-1e-17

In [18]:
female_unprivileged_score = mdss_classified.score_groups(privileged=False)
female_unprivileged_score

-1e-17

It appears there is no multiplicative increase in the odds for males thus no bias towards males and the bias score is negligible. Similarly there is no multiplicative decrease in the odds for females.

### Bias scan
We get the bias score for the apriori defined subgroup but assuming we had no prior knowledge 
about the predictive bias and wanted to find the subgroups with the most bias, we can apply bias scan to identify the priviledged and unpriviledged groups. The privileged argument is not a reference to a group but the direction for which to scan for bias.

In [19]:
privileged_subset = mdss_classified.bias_scan(penalty=0.5, privileged=True)
unprivileged_subset = mdss_classified.bias_scan(penalty=0.5, privileged=False)

In [20]:
print(privileged_subset)
print(unprivileged_subset)

({'sex': [1.0], 'age_cat': [1.0]}, 4.6919771478621115)
({'race': [0.0], 'sex': [0.0], 'c_charge_degree': [0.0], 'age_cat': [1.0]}, 4.843225441671855)


In [21]:
assert privileged_subset[0]
assert unprivileged_subset[0]

We can observe that the bias score is higher than the score of the prior groups. These subgroups are guaranteed to be the highest scoring subgroup among the exponentially many subgroups.

For the purposes of this example, the logistic regression model systematically overestimates the recidivism risk of individuals in the `Non-caucasian`, `Male`, `Misdemeanor`, `25 - 45` subgroup whereas individuals belonging to the `Female`, `25 - 45` are assigned a higher risk than is actually observed. We refer to these subgroups as the `detected privileged group` and `detected unprivileged group` respectively.

We can create another srtuctured dataset using the new groups to compute other dataset metrics.  

In [22]:
protected_attr_names = set(privileged_subset[0].keys()).union(set(unprivileged_subset[0].keys()))
dataset_orig_test.protected_attribute_names = list(protected_attr_names)
dataset_bias_test.protected_attribute_names = list(protected_attr_names)

protected_attr = np.where(np.isin(dataset_orig_test.feature_names, list(protected_attr_names)))[0]

dataset_orig_test.protected_attributes = dataset_orig_test.features[:, protected_attr]
dataset_bias_test.protected_attributes = dataset_bias_test.features[:, protected_attr]

In [23]:
# converts from dictionary of lists to list of dictionaries
a = list(privileged_subset[0].values())
subset_values = list(itertools.product(*a))

detected_privileged_groups = []
for vals in subset_values:
    detected_privileged_groups.append((dict(zip(privileged_subset[0].keys(), vals))))
    
a = list(unprivileged_subset[0].values())
subset_values = list(itertools.product(*a))

detected_unprivileged_groups = []
for vals in subset_values:
    detected_unprivileged_groups.append((dict(zip(unprivileged_subset[0].keys(), vals))))

In [24]:
metric_bias_test = BinaryLabelDatasetMetric(dataset_bias_test, 
                                             unprivileged_groups=detected_unprivileged_groups,
                                             privileged_groups=detected_privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" 
      % metric_bias_test.mean_difference())

Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.068606


It appears the detected privileged group have a higher risk of recidivism than the unprivileged group.

As noted in the paper, predictive bias is different from predictive fairness so there's no the emphasis in the subgroups having comparable predictions between them. 
We can investigate the difference in what the model predicts vs what we actually observed as well as the multiplicative difference in the odds of the subgroups.

In [25]:
to_choose = df[privileged_subset[0].keys()].isin(privileged_subset[0]).all(axis=1)
temp_df = df.loc[to_choose]

In [26]:
group_obs = temp_df['observed'].mean()
group_prob = 1-temp_df['probabilities'].mean()

"Our detected priviledged group has a size of {}, we observe {:.2%} as the average risk of recidivism, but our model predicts {:.2%}"\
.format(len(temp_df), group_obs, group_prob)

'Our detected priviledged group has a size of 188, we observe 48.40% as the average risk of recidivism, but our model predicts 36.98%'

In [27]:
odds_mul = (group_obs / (1 - group_obs)) / (group_prob /(1 - group_prob))
"This is a multiplicative increase in the odds by {:.3f}".format(odds_mul)

'This is a multiplicative increase in the odds by 1.599'

In [28]:
assert odds_mul > 1

In [29]:
to_choose = df[unprivileged_subset[0].keys()].isin(unprivileged_subset[0]).all(axis=1)
temp_df = df.loc[to_choose]

In [30]:
group_obs = temp_df['observed'].mean()
group_prob = 1-temp_df['probabilities'].mean()

"Our detected unpriviledged group has a size of {}, we observe {:.2%} as the average risk of recidivism, but our model predicts {:.2%}"\
.format(len(temp_df), group_obs, group_prob)

'Our detected unpriviledged group has a size of 165, we observe 40.61% as the average risk of recidivism, but our model predicts 54.30%'

In [31]:
odds_mul = (group_obs / (1 - group_obs)) / (group_prob /(1 - group_prob))
"This is a multiplicative decrease in the odds by {:.3f}".format(odds_mul)

'This is a multiplicative decrease in the odds by 0.575'

In [32]:
assert odds_mul < 1

In summary, this notebook demonstrates the use of bias scan to identify subgroups with significant predictive bias, as quantified by a likelihood ratio score, using subset scanning. This allows consideration of not just subgroups of a priori interest or small dimensions, but the space of all possible subgroups of features.
It also presents opportunity for a kind of bias mitigation technique that uses the multiplicative odds in the over-or-under estimated subgroups to adjust for predictive fairness.